In [1]:
import numpy as np
%load_ext Cython

In [2]:
# configurations
NDECKS: int = 8
NPLAYERS: int = 4 # affects how many cards you may be able to count
BURNPROB: float = 1.0

In [26]:
np.byte

numpy.int8

In [31]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""

# We use bytestrings as our dtype so we can have a Cython memview over the numpy card stream using C byte chars types
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A'], dtype='S2') # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'], dtype='S2')
deck = np.array([np.char.add(cards, suit) for suit in suits], dtype='S2')
stream = np.repeat(deck, NDECKS)

In [4]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(stream)

8.37 µs ± 534 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards, values))
%timeit np.array([dict_map[x[0]] for x in stream])
# %timeit np.vectorize(dict_map.get)(stream)

145 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
%%cython
import numpy as np
cimport cython 

# This is the equivalent of 'U2' it seems, compiler accepts it
cdef packed struct card_dtype_struct: 
    char card
    char suit

""" 
C uses ascii integer representation for characters
Numeric characters in our stream will have an ascii value less than ascii of 'a'
English characters will have an ascii value greater than ascii of 'a'
"""
cdef int ASCII_A = 65 

@cython.boundscheck(False)
@cython.wraparound(False)
def play_stream(card_dtype_struct[:] stream not None):
    cdef Py_ssize_t i = 0  # track dealt out index
    


In [32]:
play_stream(stream)

In [12]:
stream[:100]

array(['JH', '2C', '3S', 'AH', '6S', '9C', 'KS', '9C', 'KS', 'KD', '8C',
       '4C', 'KD', '7D', 'TC', '4D', '9D', '4S', '3H', '4D', 'JS', '7D',
       '8C', '7D', 'TD', 'AS', 'KS', 'AS', '8C', 'TS', '2C', 'AD', '2C',
       '8C', '9C', '9D', '8C', '6H', '2C', '3C', '5H', '2S', '2H', '8H',
       'AH', '5C', 'TH', '4H', 'AS', 'KH', '9S', '3C', '6S', '6C', 'QS',
       'KD', '9D', 'TD', 'QS', 'KC', 'TC', '3D', '5S', '7S', '2S', '2H',
       'JS', 'TS', '6H', '6D', '2D', '4C', '4H', 'QS', '3S', '4H', 'TS',
       '8D', 'KD', '4D', 'JS', '4H', '9C', '9C', 'AD', '6S', '7H', '9H',
       '7C', 'KS', '6H', 'QD', 'JH', '7H', 'JD', '5D', 'AC', 'TS', '6D',
       '4D'], dtype='<U2')

In [13]:
np.dtype(stream[0])

TypeError: data type 'JH' not understood